In [10]:
#collect here all imports and installs
import cv2
import os
import subprocess
import numpy as np
import pandas as pd
import time



#see what we need from these 
import pandas as pd
import numpy as np
import re
import urllib.parse


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn import metrics
from sklearn.metrics import accuracy_score 



import matplotlib.pyplot as plt
from sklearn import tree
from six import StringIO  
from IPython.display import Image  
import pydotplus
from matplotlib.colors import LinearSegmentedColormap
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [14]:
# assign directory
directory = 'videos'

# Start the timer
start_time = time.time()
df = CreateFeatureTable(directory)
print(df)
# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Table created in: {elapsed_time} seconds")

Table created in: 1081.4419901371002 seconds
             Name        Bitrate  width  height  spatial_info  temporal_info  \
0  lessmotion.MOV    8693.462433   1920    1080     88.410443     118.711078   
1      motion.MOV    8711.101114   1920    1080     62.280166     125.082111   
2           1.mp4   22108.293169   4096    2160    111.269295     111.165728   
3         raw.y4m   36460.374825    352     288     76.864047      93.444637   
4  testscenes.MOV    5156.035241   1920    1080     12.568396     126.699359   
5          1c.mp4   12237.673033   4096    2160    104.461582     105.318450   
6      ProRes.MOV  739346.248398   3840    2160     61.760261     123.150644   
7       raw_c.mp4     520.186994    352     288     76.631422     114.270600   
8     ProResc.mp4   80927.635266   3840    2160     61.564258     121.886149   

   average_intensity  average_brightness  average_contrast  average_sharpness  
0         175.183531          175.183531         69.286606         192.669

In [ ]:
#without motion score
#Table created in: 188.67073893547058 seconds
    
#with spatial and temporal info
#Table created in: 629.0842838287354 seconds

#with motion score
#Table created in: 3010.119035959244 seconds


In [8]:
#Run to initiate functions 


def CreateFeatureTable(directory):
    #Create Pandas table 
    
    # Start the timer
    start_time = time.time()
    # Create the pandas DataFrame with column name is provided explicitly
    df = pd.DataFrame()
    
    filenames = []
    bitrates = [] 
    width = []
    height = []
    fps = [] #frames Per Second, should we round?
    
    average_intensity = []

    average_brightness = []
    average_contrast = [] 
    average_sharpness = []
    motion_score = []
    spatial_info = []
    temporal_info = []


    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            filenames.append(f.replace('videos/', ''))
            b = GetBitrate(f)
            bitrates.append(b)
            #average_intensity.append(AverageSpatialValue(f))
            #SP = AverageSpatialInformation(f)
            #average_brightness.append(SP[0])
            #average_contrast.append(SP[1])
            #average_sharpness.append(SP[2])
            #motion_score.append(calculateMotionScore(f))
            
            properties = get_video_properties(f)
            width.append(properties[0])
            height.append(properties[1])
            fps.append(properties[2])
            spatial_info.append(calculate_spatial_information(f))
            temporal_info.append(calculate_temporal_information(f))
            

    df = df.assign(Name=filenames,
                   Bitrate=bitrates,
                   width=width,
                   height=height,
                   spatial_info=spatial_info,
                   temporal_info=temporal_info,
                   #average_intensity=average_intensity,
                   #average_brightness = average_brightness,
                   #average_contrast = average_contrast,
                   #average_sharpness = average_sharpness,
                   #motion_score = motion_score,
                   
                   fps=fps #why is not working??
                  )
    # End the timer
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    print(f"Table created in: {elapsed_time} seconds")
  
    return df
    
#Find video properties 

def get_video_properties(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the video width and height
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Get the frames per second (FPS)
    fps = video.get(cv2.CAP_PROP_FPS)

    # Release the video file
    video.release()

    return width, height, fps


#Calculate the bitrate 

def GetBitrate(video_path):
    
    # Start the timer
    #start_time = time.time()
    
    # Get video file size in bytes
    file_size = os.path.getsize(video_path)

    # Get video duration in seconds using FFprobe
    ffprobe_cmd = f'ffprobe -i "{video_path}" -show_entries format=duration -v quiet -of csv="p=0"'
    result = subprocess.run(ffprobe_cmd, capture_output=True, text=True, shell=True)
    duration = float(result.stdout.strip())

    # Calculate the estimated bitrate in kilobits per second (kbps)
    bitrate = (file_size * 8) / (duration * 1000)
    
    # End the timer
    #end_time = time.time()

    # Calculate the elapsed time
    #elapsed_time = end_time - start_time

    #print(f"Bitrate calculated in: {elapsed_time} seconds")
    
    return bitrate

    #print(f"Video Bitrate: {bitrate:.2f} kbps")

    
    
def calculate_spatial_information(video_path):
    video = cv2.VideoCapture(video_path)
    sobel_stds = []

    while True:
        ret, frame = video.read()

        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply the Sobel filter to the luminance component
        sobel_x = cv2.Sobel(gray_frame, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray_frame, cv2.CV_64F, 0, 1, ksize=3)
        sobel = np.sqrt(sobel_x**2 + sobel_y**2)

        # Calculate the standard deviation of the Sobel filter
        sobel_std = np.std(sobel)

        sobel_stds.append(sobel_std)

    video.release()

    # Calculate the maximum standard deviation
    max_std = max(sobel_stds)

    return max_std

def calculate_temporal_information(video_path):
    video = cv2.VideoCapture(video_path)
    motion_diffs = []

    ret, prev_frame = video.read()

    while True:
        ret, curr_frame = video.read()

        if not ret:
            break

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        # Calculate the motion difference between frames
        motion_diff = curr_gray - prev_gray

        # Calculate the standard deviation of the motion difference
        motion_std = np.std(motion_diff)

        motion_diffs.append(motion_std)

        prev_frame = curr_frame

    video.release()
    # Calculate the maximum standard deviation
    max_std = max(motion_diffs)

    return max_std


#WHAT IT GIVES US?

#Average Spatial Value

def AverageSpatialValue(video_path):
    # Start the timer
    #start_time = time.time()
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is successfully opened
    if not cap.isOpened():
        print("Error opening video file")
        exit()

    # Initialize variables
    frame_count = 0
    total_intensity = 0

    # Read and process frames
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # If the frame was not read successfully, then we have reached the end of the video
        if not ret:
            break

        # Calculate the average pixel intensity of the frame
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        intensity = np.mean(gray_frame)

        # Accumulate the total intensity
        total_intensity += intensity

        # Display the frame with intensity value (optional)
        #cv2.putText(frame, f"Intensity: {intensity:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        #cv2.imshow('Frame with Intensity', frame)
        #if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        #    break

        # Increment frame count
        frame_count += 1

    # Calculate the average spatial value
    average_intensity = total_intensity / frame_count

    # Release the video capture object and close any open windows
    cap.release()
    cv2.destroyAllWindows()
    
    # End the timer
    #end_time = time.time()

    # Calculate the elapsed time
    #elapsed_time = end_time - start_time

    #print(f"Average Spatial Value calculated in: {elapsed_time} seconds")
    
    return average_intensity
    #print(f"Average Spatial Value: {average_intensity}")
    
    
    
#Calculate average spatial information


import cv2
def AverageSpatialInformation(video_path):

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Initialize variables to store spatial information
    frame_count = 0
    total_brightness = 0.0
    total_contrast = 0.0
    total_sharpness = 0.0

    # Iterate over the frames
    while True:
        # Read the next frame
        ret, frame = cap.read()

        # If no more frames, break the loop
        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate brightness (average pixel intensity)
        brightness = gray_frame.mean()
        total_brightness += brightness
        #print('calculate brightness')
        # Calculate contrast (standard deviation of pixel intensities)
        contrast = gray_frame.std()
        total_contrast += contrast
        #print('calculate contrast')
        # Calculate sharpness (Laplacian variance)
        laplacian_var = cv2.Laplacian(gray_frame, cv2.CV_64F).var()
        total_sharpness += laplacian_var
        #print('calculate sharpness')

        # Increment frame count
        frame_count += 1

    # Release the video capture
    cap.release()

    # Calculate average spatial information
    average_brightness = total_brightness / frame_count
    average_contrast = total_contrast / frame_count
    average_sharpness = total_sharpness / frame_count
    
    return average_brightness, average_contrast, average_sharpness

    #print(frame_count)
    # Print the spatial information measures
    print(f"Average Brightness: {average_brightness}")
    print(f"Average Contrast: {average_contrast}")
    print(f"Average Sharpness: {average_sharpness}")



SyntaxError: invalid syntax (<ipython-input-8-7602ae877c52>, line 64)

In [ ]:
#TAKES TOO LONG

#Analyse motion
#Returns video Motion Score


def calculateMotionScore(video_path):

    # Start the timer
    #start_time = time.time()

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Read the first frame
    ret, prev_frame = cap.read()

    # Convert the frame to grayscale
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Initialize motion vectors
    motion_vectors = []

    # Set parameters for Farneback optical flow
    params = dict(pyr_scale=0.5, levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    # Iterate over the frames
    while True:
        # Read the next frame
        ret, frame = cap.read()

        # If no more frames, break the loop
        if not ret:
            break

        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow using Farneback algorithm
        flow = cv2.calcOpticalFlowFarneback(prev_frame_gray, frame_gray, None, **params)

        # Calculate the magnitude of motion vectors
        magnitude = np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2)

        # Calculate the average magnitude
        average_magnitude = np.mean(magnitude)

        # Store the average magnitude as a motion value
        motion_vectors.append(average_magnitude)

        # Set the current frame as the previous frame for the next iteration
        prev_frame_gray = frame_gray.copy()

    # Release the video capture
    cap.release()

    # Calculate the overall motion score
    motion_score = np.mean(motion_vectors)



    # End the timer
    #end_time = time.time()

    # Calculate the elapsed time
    #elapsed_time = end_time - start_time

    #print(f"Elapsed Time too find motion score: {elapsed_time} seconds")

    #print(f"Motion Score: {motion_score}")
    return motion_score


In [72]:

#for working with csv from Leon 

path_to_dataset = 'features.csv'
path_to_labels = 'bitrates.csv'

#upload 
#upload data
df_features = pd.read_csv(path_to_dataset)
df_labels = pd.read_csv(path_to_labels)
#cut names of videos
#this works only for current table, have to think how we work with names later

#df
#df_labels big buck bunny 720p24 Scene 049.mp4
df_features['Name'] = df_features['Name'].str[-7:]
df_labels['Name'] = df_labels['Name'].str[-7:]

#drop first column df_labels
df_labels = df_labels.drop(columns=['Unnamed: 0'])


#merge tables 
df = pd.merge(df_features, df_labels, on=['Name'])
df = df.rename(columns={'Bitrate_y': 'Bitrate_Label'}) 
df.head()


,Name,Bitrate_x,width,height,spatial_info,temporal_info,average_intensity,average_brightness,average_contrast,average_sharpness,Bitrate_Label
0,002.mp4,78676.570839,1280,720,84.144972,121.727693,121.837159,121.837159,61.129357,922.296702,2275.346517
1,016.mp4,35535.435877,1280,720,59.938475,113.200446,108.749931,108.749931,57.194242,1028.256694,1421.828734
2,017.mp4,38315.829029,1280,720,29.277798,109.325765,187.289447,187.289447,39.956619,25.712357,1042.835766
3,003.mp4,36179.684015,1280,720,48.599192,118.055684,158.708454,158.708454,46.707699,98.187098,985.768549
4,015.mp4,33525.657021,1280,720,44.378206,100.921918,159.687033,159.687033,61.429923,101.481319,725.144206


In [77]:
#DECISION TREE

# import the regressor
from sklearn.tree import DecisionTreeRegressor 


#assume df is a table with features and labels 
#where label is in the last column called Bitrate_Label

#names of the columns with features, 
#can also just take all columns apart from first with name and last with label

#we can leave some out of course
list_of_features = ['Bitrate_x', 'spatial_info','temporal_info', 'average_intensity',
                   'average_brightness', 'average_contrast', 'average_sharpness']

X = df[list_of_features] #Features
y = df.Bitrate_Label #Labels



#split the data into train and test 

#Split the data into training set and test set
# 67% training and 33% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 


# create a regressor object
regressor = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
regressor.fit(X_train, y_train)

# Predict

y_predicted = regressor.predict(X_test)


#to compare the result we have to set some treshhold 
#like +-5% of abweichung in bitrate for prediction 
compare = pd.DataFrame()
compare= compare.assign(test =y_test, predicted = y_predicted)


In [78]:
compare

,test,predicted
13,1953.028908,1752.416025
39,1906.480403,1674.667095
30,1281.850457,1055.434705
45,1971.823239,1319.007533
17,2299.811207,2275.346517
48,1240.671329,1319.007533
26,1322.270383,1838.300600
25,2341.947224,2377.380565
32,670.499928,886.716199
19,2001.957490,1653.285318
